**А вот и наш первый минипроект :)**

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time
bookings = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv', sep=';')
bookings_head=bookings[:7]

Как вы уже могли заметить, при сборе датасета произошла небольшая ошибка, и названия колонок были записаны в неудобном для дальнейшей работы формате. Исправьте ситуацию: приведите названия столбцов к нижнему регистру и замените пробелы на знак нижнего подчёркивания. Например: Is Canceled -> is_canceled

In [3]:
bookings.columns = bookings.columns.str.lower()
bookings.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [5]:
bookings.head(2)

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,stays_total_nights,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01


Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга.

Сколько подобных наблюдений встретилось в датасете?

In [6]:
bookings.query('assigned_room_type != reserved_room_type').shape[0]

14917

In [7]:
bookings.query('arrival_date_year == 2016').arrival_date_month.value_counts()
#bookings.query('arrival_date_year == 2017').arrival_date_month.value_counts()

October      6203
May          5478
April        5428
September    5394
June         5292
August       5063
March        4824
July         4572
November     4454
February     3891
December     3860
January      2248
Name: arrival_date_month, dtype: int64

В 2016 чаще всего бронировали отель на октябрь
В 2017 чаще всего оформляли бронирование на май

Сгруппируйте данные по годам, а затем проверьте, на какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 

In [8]:
(
    bookings.query('hotel == "City Hotel" and is_canceled == 1')
      .groupby('arrival_date_year')
      .arrival_date_month
      .value_counts()
)

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

Посмотрите на числовые характеристики трёх колонок: adults, children и babies. Какая из них имеет наибольшее среднее значение?

In [9]:
bookings[['adults', 'children', 'babies']].mean().idxmax()

'adults'

Создайте колонку total_kids, объединив столбцы children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим?

City hotel – отель находится в городе
Resort hotel – отель курортный
В качестве ответа укажите наибольшее среднее total_kids, округлив до 2 знаков после точки.

In [13]:
bookings['total_kids'] = bookings['children'] + bookings['babies']

In [23]:
bookings.groupby('hotel').agg({'total_kids': 'mean'}).round(2).max()

total_kids    0.14
dtype: float64

Не все бронирования завершились успешно (is_canceled), поэтому можно посчитать, сколько клиентов было потеряно в процессе. Churn rate (отток, коэффициент оттока) – это процент подписчиков (например, на push-уведомления от сайта), которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Иными словами, представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах.

Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Далее проверьте, среди какой группы пользователей показатель оттока выше. 

В качестве ответа укажите наибольший %, округленный до 2 знаков после точки (то есть доля 0.24563 будет 24.56% и в ответ пойдёт 24.56)

In [33]:
bookings['has_kids'] = bookings.total_kids > 0 
bookings['has_kids']

0         False
1         False
2         False
3         False
4         False
          ...  
119385    False
119386    False
119387    False
119388    False
119389    False
Name: has_kids, Length: 119390, dtype: bool

In [40]:
no_kids_churn_rate  = bookings.query('is_canceled == 1 and has_kids == False').shape[0] / bookings.query('has_kids == False').shape[0]
print( round(no_kids_churn_rate * 100, 2))

37.22


In [41]:
has_kids_churn_rate  = bookings.query('is_canceled == 1 and has_kids == True').shape[0] / bookings.query('has_kids == True').shape[0]
print( round(has_kids_churn_rate * 100, 2))

34.92
